In [ ]:
import numpy as np
import numpy.matlib
import pandas as pd
import matplotlib.pyplot as plt

import PHD.cnv_model_utils as u

import sys
import time
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler

from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import Lasso
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.metrics import roc_curve, precision_recall_curve
from sklearn.model_selection import cross_val_score,StratifiedKFold


%matplotlib inline

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
# https://pytorch.org/docs/stable/notes/randomness.html
seed = u.get_seed()
print('Seed = ', seed)
u.set_all_seeds(seed)

---
---
---

In [ ]:
# PATH = "D:/CANCER BIOLOGY/DATASET/CNV/TCGA/FROM cBioPortal/"
PATH = 'D:/CANCER BIOLOGY/DATASET/CNV/TCGA/FROM Xena/'

In [ ]:
# df_final = pd.read_csv(PATH+'CNV_CBIOPORTAL_DATA_PREPROCESSED.gz', sep='\t', compression='gzip')
df_final = pd.read_csv(PATH+'TCGA_XENA_LUAD_LUSC_CNV_dataset_preprocessed.gz', sep='\t', compression='gzip')
df_final = df_final.sample(frac=1, random_state=seed).reset_index(drop=True)
labels = list(df_final['label'])
df_final.drop(columns=['label'], axis=1, inplace=True) ## drop column sample_id and label
columns = list(df_final.columns)

In [ ]:
df_final

In [ ]:
xtrain = df_final.to_numpy()
ytrain = labels

In [ ]:
device = torch.device('cuda' if  torch.cuda.is_available() else 'cpu')
device

---
---
---

In [ ]:
class CNV(Dataset):
    
    def __init__(self, X, y):
        self.X = X
        self.y = y
        print("No. of samples in dataset = ", len(self.X))
        print("No. of unique labels in dataset = ", set(self.y))
                
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [ ]:
class Network(nn.Module):
    
    def __init__(self, input_dim, output_dim, exe='lasso'):
        super().__init__()
        
        self.exe = exe
        
        self.layer1 = nn.Linear(input_dim, 4096)
        self.norm1 = nn.Dropout(0.4)
        self.relu1 = nn.ReLU()
        
        self.layer2 = nn.Linear(4096, 2048)
        self.norm2 = nn.Dropout(0.3)
        self.relu2 = nn.ReLU()
        
        self.layer3 = nn.Linear(2048, 1024)
        self.norm3 = nn.Dropout(0.2)
        self.relu3 = nn.ReLU()
        
        self.layer4 = nn.Linear(1024, 512)
        self.norm4 = nn.Dropout(0.1)
        self.relu4 = nn.ReLU()
        
        self.layer5 = nn.Linear(512, output_dim)
        self.relu5 = nn.ReLU()
        
        self.layer6 = nn.Linear(output_dim, 2)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, y):
        
        importances = []
        
        layer1 = self.layer1(x)
        norm1 = self.norm1(layer1)
        relu1 = self.relu1(norm1)
        importances.append(compute_lasso(relu1, y) if self.exe=='lasso' else compute_xgb(relu1, y))
        
        layer2 = self.layer2(relu1)
        norm2 = self.norm2(layer2)
        relu2 = self.relu2(norm2)
        importances.append(compute_lasso(relu2, y) if self.exe=='lasso' else compute_xgb(relu2, y))
        
        layer3 = self.layer3(relu2)
        norm3 = self.norm3(layer3)
        relu3 = self.relu3(norm3)
        importances.append(compute_lasso(relu3, y) if self.exe=='lasso' else compute_xgb(relu3, y))
        
        layer4 = self.layer4(relu3)
        norm4 = self.norm4(layer4)
        relu4 = self.relu4(norm4)
        importances.append(compute_lasso(relu4, y) if self.exe=='lasso' else compute_xgb(relu4, y))
        
        layer5 = self.layer5(relu4)
        relu5 = self.relu5(layer5)
        importances.append(compute_lasso(relu5, y) if self.exe=='lasso' else compute_xgb(relu5, y))
        
        layer6 = self.layer6(relu5)
        sigmoid = self.sigmoid(layer6)
        importances.append(compute_lasso(sigmoid, y) if self.exe=='lasso' else compute_xgb(sigmoid, y))
        
        return sigmoid, importances
        

In [ ]:
'''

Utility functions:

1. compute_lasso()
2. compute_xgb()
3. nudge_weights()
4. train_model()
5. validate_model()

'''

## effect of alpha in LASSO regularization
## https://chrisalbon.com/code/machine_learning/linear_regression/effect_of_alpha_on_lasso_regression/
def compute_lasso(samples, labels):
    c_lasso = Lasso(alpha=0.0001, random_state=seed)
    c_lasso.fit(samples.detach().cpu().numpy(), labels.detach().cpu().numpy())
    return c_lasso.coef_

def compute_xgb(samples, labels):
    c_xgb = XGBClassifier(max_depth=100, random_state=seed, verbosity=0)
    c_xgb.fit(samples.detach().cpu().numpy(), labels.detach().cpu().numpy())
    return c_xgb.feature_importances_

def nudge_weights(model, importances):
    i = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            important_tensor = torch.tensor([importances[i]]).T.to(device)
            min_val = torch.min(param.grad)
            i+=1
            if min_val != 0:
                updation = torch.log(torch.abs(min_val)).to(device)
                important_tensor*=(10**updation)
                param.grad += important_tensor
    
def train_model(device, loader, model, optimizer, loss_fn):
    model.train()
    for samples, labels in loader:
        samples = samples.to(device)
        samples = samples.float()
        labels = labels.to(device)
        
        ## ---------------------------------------------------------------------------- ##
        
        optimizer.zero_grad()  ## zer the previously computed gradients and start afresh
        preds, importances = model(samples, labels)  ## train the model
        cost = loss_fn(preds, labels)  ## compute cost
        cost.backward()  ## compute gradients
        nudge_weights(model, importances)  ## update the weights using computed importances
        optimizer.step()  ## perform a single step of optimizer to update the weights
        
        ## ---------------------------------------------------------------------------- ##
        
        scores, predictions = torch.max(preds, 1)
        return (predictions == labels).sum().item(), cost.item()
    

def validate_model(device, loader, model):
    model.eval()
    with torch.no_grad():
        for samples, labels in loader:
            samples = samples.to(device)
            samples = samples.float()
            labels = labels.to(device)

            preds, _ = model(samples, labels)

            scores, predictions = torch.max(preds, 1)
            fpr, tpr, _ = roc_curve(labels.detach().cpu(), predictions.detach().cpu())
            auroc = roc_auc_score(labels.detach().cpu(), predictions.detach().cpu())
            auprc = average_precision_score(labels.detach().cpu(), predictions.detach().cpu())
            precision, recall, _ = precision_recall_curve(labels.detach().cpu(), predictions.detach().cpu())
            return (predictions == labels).sum().item(), fpr, tpr, precision, recall, auroc, auprc

In [ ]:
batch_size = 64
input_dim = xtrain.shape[1]
output_dim = 256
epochs = 50
learn = 1e-4
dataset = CNV(df_final.to_numpy(), labels)

---
---
---

### USING LASSO

In [ ]:
'''

1. EXECUTION USING LASSO REGRESSION

Perform K-Fold cross validation using SubsetSampler
feature of PyTorch. Try to follow this approach from now on.
Essential links:
https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-pytorch.md
https://medium.com/dataseries/k-fold-cross-validation-with-pytorch-and-sklearn-d094aa00105f
'''

mean_train_score = []
mean_fpr = []
mean_tpr = []
mean_precision = []
mean_recall = []
mean_auroc = []
mean_auprc = []

mean_valid_score = []
k = 10
u.set_all_seeds(seed)
kfold = StratifiedKFold(n_splits=k, random_state=seed, shuffle=True)

## start the k-fold
for train_ids, valid_ids in kfold.split(dataset, labels):
    
    train_sampler = SubsetRandomSampler(train_ids)
    valid_sampler = SubsetRandomSampler(valid_ids)
    
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
    
    u.set_all_seeds(seed)
    model = Network(input_dim, output_dim, exe = 'lasso')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learn)
    loss_fn = nn.CrossEntropyLoss()
    
    start_time = time.time()
    ## train and validate
    for epoch in range(epochs):
        
        train_correct, final_loss = train_model(device, train_loader, model, optimizer, loss_fn)
        valid_correct, fpr, tpr, precision, recall, auroc, auprc = validate_model(device, valid_loader, model)
        if epoch % 20 == 0:
            print("Epoch: %03d/%03d | Train Acc %.3f | Valid Acc %.3f | Loss %.3f" %
                  (epoch, epochs, train_correct/batch_size, valid_correct/batch_size, final_loss))
    
    print('Time of exe:', time.time()-start_time)
    print('\n*******************\n')
    
    mean_train_score.append(train_correct/batch_size)
    mean_valid_score.append(valid_correct/batch_size)
    mean_fpr.append(fpr)
    mean_tpr.append(tpr)
    mean_precision.append(precision)
    mean_recall.append(recall)
    mean_auroc.append(auroc)
    mean_auprc.append(auprc)

print('MEAN_TRAIN_SCORE : %.3f | MEAN_VALID_SCORE : %.3f'%(np.mean(mean_train_score), np.mean(mean_valid_score)))

In [ ]:
np.mean(mean_auroc), np.mean(mean_auprc)

In [ ]:
# Plot AUROC and AUPRC curves
plt.figure(figsize=(12, 5))

# AUROC Curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='b', lw=2, label=f'Mean AUROC = {np.mean(mean_auroc):.3f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (AUROC) Curve')
plt.legend(loc='lower right')
plt.savefig('auroc_cnv.pdf', dpi=300)
plt.show()

# AUPRC Curve
plt.figure(figsize=(8, 6))
plt.plot(recall, precision, color='b', lw=2, label=f'Mean AUPRC = {np.mean(mean_auprc):.3f}')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (AUPRC) Curve')
plt.legend(loc='lower right')
plt.savefig('auprc_cnv.pdf', dpi=300)
plt.show()

In [ ]:
torch.save(model.state_dict(), PATH+"models/CUSTOM_CNV_NETWORK_lasso.kd")

### USING XGBOOST

In [ ]:
'''

2. EXECUTION USING XGBOOST

Perform K-Fold cross validation using SubsetSampler
feature of PyTorch. Try to follow this approach from now on.
Essential links:
https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-pytorch.md
https://medium.com/dataseries/k-fold-cross-validation-with-pytorch-and-sklearn-d094aa00105f
'''

mean_train_score = []
mean_auroc = []
mean_auprc = []
mean_valid_score = []
k = 10
u.set_all_seeds(seed)
kfold = StratifiedKFold(n_splits=k, random_state=seed, shuffle=True)

## start the k-fold
for train_ids, valid_ids in kfold.split(dataset, labels):
    
    train_sampler = SubsetRandomSampler(train_ids)
    valid_sampler = SubsetRandomSampler(valid_ids)
    
    train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)
    
    u.set_all_seeds(seed)
    model = Network(input_dim, output_dim, exe = 'xgb')
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learn)
    loss_fn = nn.CrossEntropyLoss()
    
    start_time = time.time()
    ## train and validate
    for epoch in range(epochs):
        
        train_correct, final_loss = train_model(device, train_loader, model, optimizer, loss_fn)
        valid_correct, auroc, auprc = validate_model(device, valid_loader, model)
        if epoch % 20 == 0:
            print("Epoch: %03d/%03d | Train Acc %.3f | Valid Acc %.3f | Loss %.3f" %
                  (epoch, epochs, train_correct/batch_size, valid_correct/batch_size, final_loss))
    
    print('Time of exe:', time.time()-start_time)
    print('\n*******************\n')
    
    mean_train_score.append(train_correct/batch_size)
    mean_valid_score.append(valid_correct/batch_size)

print('MEAN_TRAIN_SCORE : %.3f | MEAN_VALID_SCORE : %.3f'%(np.mean(mean_train_score), np.mean(mean_valid_score)))

In [ ]:
torch.save(model.state_dict(), PATH+"models/CUSTOM_CNV_NETWORK_xgb.kd")

In [ ]:
from scipy.io.wavfile import read

fs, data = read('PHD/alert.wav', mmap=True)  # fs - sampling frequency
data = data.reshape(-1, 1)
import sounddevice as sd
sd.play(data, 44100)

---
---
---